<a href="https://colab.research.google.com/github/Perfect-Cube/Volkswagon-imobilothon-4.0/blob/main/BatteryCooling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda, Layer, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

# Larger simulated dataset for Indian climate and driving conditions
data = pd.DataFrame({
    'route': np.random.choice(['urban', 'highway', 'hilly', 'coastal'], size=5000),
    'ambient_temp': np.random.uniform(10, 50, size=5000),  # Expanded to cover high temperatures in India
    'speed': np.random.uniform(0, 120, size=5000),
    'motor_workload': np.random.uniform(0, 1, size=5000),
    'battery_temp': np.random.uniform(20, 80, size=5000)
})

# Encode routes for different Indian regions
data = pd.get_dummies(data, columns=['route'])

# Scale features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(data[['ambient_temp', 'speed', 'motor_workload', 'battery_temp']])

# Define Variational Autoencoder (VAE) for dynamic cooling prediction
latent_dim = 2
inputs = Input(shape=(scaled_features.shape[1],))
h = Dense(64, activation='relu')(inputs)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

# Sampling function using Keras Lambda layer
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=0.1)
    return z_mean + K.exp(z_log_var / 2) * epsilon

z = Lambda(sampling)([z_mean, z_log_var])

# Decoder part
decoder_h = Dense(64, activation='relu')
decoder_mean = Dense(scaled_features.shape[1])  # Output back to original feature size (4)
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

# Define a custom VAE loss layer
class VAELossLayer(Layer):
    def call(self, inputs, **kwargs):
        x = inputs[0]
        x_decoded_mean = inputs[1]
        z_mean = inputs[2]
        z_log_var = inputs[3]

        reconstruction_loss = tf.reduce_mean(tf.square(x - x_decoded_mean))
        kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        vae_loss = K.mean(reconstruction_loss + kl_loss)

        self.add_loss(vae_loss)
        return x_decoded_mean

# Apply the custom loss layer
outputs = VAELossLayer()([inputs, x_decoded_mean, z_mean, z_log_var])
vae = Model(inputs, outputs)

# Compile and train the VAE
vae.compile(optimizer='adam')
vae.fit(scaled_features, scaled_features, epochs=50, batch_size=64, validation_split=0.2)

# Cooling methods
def air_cooling():
    print("Air cooling: Suitable for mild loads, typically in urban areas or moderate climates.")

def fan_cooling():
    print("Fan cooling: Ideal for higher loads in variable Indian climates.")

def liquid_cooling():
    print("Liquid cooling: Optimal for high-performance needs in extreme heat conditions.")

# Cooling selection logic for Indian conditions
def select_cooling_method(predicted_temp_increase, ambient_temp, speed, route):
    """
    Selects cooling method based on predicted temperature and conditions for Indian environments.
    """
    if predicted_temp_increase > 70 or (ambient_temp > 40 and route in ['hilly', 'urban']):
        liquid_cooling()
    elif predicted_temp_increase > 50 or (ambient_temp > 30 and speed > 80):
        fan_cooling()
    else:
        air_cooling()

# Scenario generation with Indian standards
def generate_cooling_scenario(input_data):
    # Prepare input data similar to training data
    input_df = pd.DataFrame([input_data])  # Create a DataFrame from the input dictionary
    input_df = pd.get_dummies(input_df, columns=['route'])  # One-hot encode route

    # Ensure all columns from training data are present in input_df
    missing_cols = set(data.columns) - set(input_df.columns)
    for col in missing_cols:
        input_df[col] = 0  # Add missing columns with 0 values

    # Scale the input data using the same scaler
    scaled_input = scaler.transform(input_df[['ambient_temp', 'speed', 'motor_workload', 'battery_temp']])

    # Now use the scaled input for prediction
    generated_data = vae.predict(scaled_input)
    predicted_temp_increase = generated_data[0, -1]

# Example input for Indian scenario
input_data_example = {'ambient_temp': 42, 'speed': 90, 'route': 'hilly'}
generate_cooling_scenario(input_data_example)


Epoch 1/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 6s 49ms/step - loss: 1.0104 - val_loss: 0.8668
Epoch 2/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7985 - val_loss: 0.6944
Epoch 3/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6690 - val_loss: 0.6542
Epoch 4/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6393 - val_loss: 0.6515
Epoch 5/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6339 - val_loss: 0.6490
Epoch 6/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6347 - val_loss: 0.6468
Epoch 7/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6343 - val_loss: 0.6489
Epoch 8/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6365 - val_loss: 0.6423
Epoch 9/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6261 - val_loss: 0.6435
Epoch 10/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6335 - val_loss: 0.6414
Epoch 11/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6218 - val_loss: 0.6438
Epoch 12/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6149 - val_l